# Finding the Rotation Curve of an Asteroid or Comet with TESScut and lightkurve
---

## Learning Goals

By the end of this tutorial, you will:
- Understand what a cutout of SOMETHING SOMETHING is?
- Use the TESScut API to find a cutout of an asteroid or comet by name.
- Plot the lightcurve using lightkurve
- Find the rotation period of the asteroid using lightkurve

## Introduction
Intro ideas...
- Asteroids and comets and moving targets, they won't have a fixed RA and Dec. But built-in tools can be searched by object name or ID understood by [JPL Horizon ephemerades interface](https://ssd.jpl.nasa.gov/horizons/app.html#/)
- "While its primary mission avoids the vicinity of the ecliptic plane by approximately six degrees, the scale height of the solar system debris disk is large enough to place various small body populations in the field of view." https://ui.adsabs.harvard.edu/abs/2020ApJS..247...26P/abstract <- They have MAST data (TSSYS-DR1) and published lightcurves and periods. We'll get the TESS cutout by object name and compare.
- What is a CUTOUT, what is an FFI TIME SERIES. FFI = full-frame images, as opposed to target pixel files (TPFs) https://heasarc.gsfc.nasa.gov/docs/tess/data-products.html 

The workflow for this notebook consists of:
    THE TOC

## Imports

- *numpy* to handle array functions
- *astropy.io fits* for acccessing FITS files
- *astropy.table Table* for creating tidy tables of the data
- *matplotlib.pyplot* for plotting data

JRK be sure to review when done-ish.

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np

from astropy.io import fits
from astropy.table import Table
from astroquery.mast import Mast
from astroquery.mast import Observations

# JRK be sure to review when done-ish.

import requests
from astroquery.mast import Tesscut
import lightkurve as lk

## Searching the TESS archive by target name using the TESScut API

You can use the [TESScut webpage](https://mast.stsci.edu/tesscut/) to make a cutout of a TESS FFI time series; for a solar system object like an asteroid or comet, be sure to click on the "Moving Target" button at the top after the words "Create cutout based on." 

To create a scriptable process for using TESScut, we can instead use the API or Application Programming Interface. We can use a script to create a unique URL that contains the same information and settings you'd input by hand on the webpage. Then we can use the python `requests` library to access that URL and deliver the results into this notebook.

The URLs for the TESScut API will all start with `https://mast.stsci.edu/tesscut/api/v0.1/` and for the case of moving targets, `moving_target/` will be appended. After `astrocut?` we will include all of the keywords we want separated by `&` signs. 
- `obj_id=hippodamia` to search by the object's name
- `x=4` for our cutout image to be 4 pixels in width
- `y=4` for our cutout image to be 4 pixels in height.

1 TESS pixel is about 21 arcseconds wide. 

JRK justify the width and height based on something useful. For now it's small b/c it's slow...

Below, we print the status code from our query; a code of 200 means success. 

In [ ]:
objname='hippodamia'

In [ ]:
url='https://mast.stsci.edu/tesscut/api/v0.1/moving_target/astrocut?obj_id='+objname+'&x=4&y=4'
r=requests.get(url)
r.status_code

In [ ]:
#r.json().keys() # THIS FREEZES!

In [ ]:
# Comparing to the .fits we get from the website... for 4x4, all sectors, but there's only 1 for Hippodamia
filename_fromwebsite='Hippodamia_1382.0576094717571-1409.3495225296479_4.0-x-4.0_astrocut.fits'
hdu1=fits.open(filename_fromwebsite)
hdu1.info()
# We do get the same thing (it seems!) as from astroquery. 

In [ ]:
# What if we try Ceres, which has multiple sectors? You get multiple files. 

## Searching the TESS archive by target name using astroquery

Built-in [MAST functionality in astroquery can also access TESScut](https://astroquery.readthedocs.io/en/latest/mast/mast.html#cutouts). 

TESS observed distinct areas of the sky called Sectors. We can use `Tesscut.get_sectors` to see which sectors inlude observations of our object. Whenever using Tesscut with a moving target, we must always set `moving_target=True` (the default is `False`). 

In [ ]:
sector_table=Tesscut.get_sectors(objectname=objname, moving_target=True)
print(sector_table)

Next we'll use `Tess.get_cutouts` to You can specify `sector=3` when requesting the cutouts, but it's not necessary. Without specifying the sector, we will receive results for all sectors available. 

JRK is that true or did I make it up. 

JRK if I chose one with multiple sectors observed, would hdulist be longer than 1? Tried with Ceres, YES, it will have length that matches the number of sectors. Also matches the number of separate fits files you'd download from the website.

In [ ]:
hdulist = Tesscut.get_cutouts(objectname=objname, moving_target=True, size=20)

In [ ]:
print('Length of hdulist =',len(hdulist))
hdulist[0].info()

The result that we get is describe in the [astrocut documentation on Path Focused Target Pixel Files](https://astrocut.readthedocs.io/en/latest/astrocut/file_formats.html#path-focused-target-pixel-files). Extension 0 is the header with metadata. Astronomers not used to working with moving targets may note that the `RA_OBJ` and `DEC_OBJ` keywords are not there!

Extension 1 is the data we most want in the form of a binary table; we'll look at tha tnext.

But first, extension 2, if present, is an image the size of the full image that the cutouts were drawn from. It's an array of 1's if the pixel was used and 0 if the pixel was not. By displaying this array as an image, we can see the trajectory of our moving target through the TESS sector. 

JRK is that right, is it the whole sector or some subset? 

In [ ]:
plt.imshow(hdulist[0][2].data)

Going back to Extension 1, this is a binary table with 1267 rows and 16 columns. The header keys `TTYPE1`, `TTYPE2`, etc. tell us the names of each column. Let's view each of them.

In [ ]:
for i in range(1,17): print(i-1,hdulist[0][1].header['TTYPE'+str(i)])

Examining the first row of this binary table, we can see that some of the columns are really arrays; these are the single cutout images we requested. 

In [ ]:
print(hdulist[0][1].data[0])

The column of index i=4 is the array of fluxes; it's exactly the size we requested. Let's view the first cutout!

In [ ]:
fig,ax=plt.subplots(1,1)
imgplot=plt.imshow(hdulist[0][1].data[0][4])
plt.colorbar()
ax.set_title(objname+' Time '+str(hdulist[0][1].data[0][0]))
print('RA: ',hdulist[0][1].data[0][14])
print('Dec: ',hdulist[0][1].data[0][15])

Let's view another cutout at a later time, say at row 500.

In [ ]:
i=500
fig,ax=plt.subplots(1,1)
imgplot=plt.imshow(hdulist[0][1].data[i][4])
plt.colorbar()
ax.set_title(objname+' Time '+str(hdulist[0][1].data[i][0]))
print('RA: ',hdulist[0][1].data[i][14])
print('Dec: ',hdulist[0][1].data[i][15])
# JRK should make the two have the same colorbar.

At this later time, the target has moved to a different RA and Dec, and also appears differently due to its rotation about its own axis.

# Attempt to get this into lightkurve

In [ ]:
hdulist[0].writeto('testfile_20x20.fits')

In [2]:
tpf = lk.TessTargetPixelFile('testfile_20x20.fits')

In [3]:
tpf.interact()

In [4]:
tpf.animate()

In [ ]:
lc=tpf.to_lightcurve()

In [ ]:
# Well this is a bunch of junk because of all the stuff in the background!!! 
lc.plot()

In [ ]:
pg=lc.to_periodogram()

In [ ]:
pg.plot()

In [ ]:
lc.fold(period=pg.period_at_max_power).scatter()

# Playing around with a non-moving target in lightkurve

Though `lightkurve` has a `search_tesscut` feature, it does not work for moving targets at the time of the writing of this tutorial. 

Just how different are these from targetpixelfiles? 

You can't do search_tesscut https://docs.lightkurve.org/reference/api/lightkurve.search_tesscut.html with moving_target=True, and you can't resolve by name to a sky position. But I'll investigate what you CAN do with a non-moving target? You get the list of dataproducts. 

Try this with a Cepheid observed by TESS... https://iopscience.iop.org/article/10.3847/1538-4365/abd4e3



In [ ]:
search_result=lk.search_tesscut('SS Gru')
print(search_result)

In [ ]:
#tpf=lk.search_targetpixelfile('Beta Dor',sector=1).download() # If don't specify by sector?
tpf=lk.search_targetpixelfile('SS Gru').download_all()

In [ ]:
tpf[0].flux.shape

In [ ]:
tpf[1].pipeline_mask # 0 and 1 are ever so slightly different. 

In [ ]:
# This works for single TPF's:
#tpf.plot(frame=0,aperture_mask=tpf.pipeline_mask)
# But for a TPF collection:
tpf.plot()

In [ ]:
# This works for single TPF's:
#tpf.interact()
# Need to specify if a collection:
tpf[1].interact()

In [ ]:
# For a single tpf:
#lc=tpf.to_lightcurve()
# For a collection, make multiple lightcurves, and combine them?
lc=tpf[0].to_lightcurve()
print(len(lc.flux))
for i in range(1,len(tpf)): 
    print(i)
    lc_new=tpf[i].to_lightcurve()
    lc=lc.append(lc_new)
    print(len(lc.flux))
        

In [ ]:
lc[1]

In [ ]:
lc.plot()

In [ ]:
pg=lc.to_periodogram()

In [ ]:
pg.plot(view='period',scale='log')
print(pg.period_at_max_power)

In [ ]:
lc.fold(period=pg.period_at_max_power).scatter()

## Exercises

## Additional Resources

## Citations

If you use `astropy` or `lightkurve` for published research, please cite the
authors. Follow these links for more information about citing `astropy` and
`lightkurve`:

TESScut citation https://ui.adsabs.harvard.edu/abs/2019ascl.soft05007B/abstract Brasseur et al. 2019

* [Citing `astropy`](https://www.astropy.org/acknowledging.html)
* [Citing `lightkurve`](http://docs.lightkurve.org/about/citing.html)

## About this Notebook

**Author:** Julia Kamenetzky, ScienceBetter Consultant  
**Keywords:**   
**Last Updated:**  
**Next Review:**  

JRK not done, and add the footer.